In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/user_visit_duration.csv')

In [4]:
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD

Using TensorFlow backend.


In [5]:
'''Benchmark Model'''

model = Sequential()
model.add(Dense(1,input_shape=(1,), activation='sigmoid')) # only difference is the activation finction

model.compile(SGD(lr=0.5), 'binary_crossentropy', metrics=['accuracy'])

In [6]:
X =df[['Time (min)']].values
y = df[['Buy']].values

model.fit(X,y,epochs=25, verbose=0)

In [7]:
y_pred = model.predict(X)
y_class_pred = y_pred>0.5

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
accuracy_score(y,y_class_pred)

0.83

# Cross evaluation

In [12]:
from keras.wrappers.scikit_learn import  KerasClassifier

In [13]:
'''build function for scikit wrapper'''

def build_logistic():
    model = Sequential()
    model.add(Dense(1,input_shape=(1,), activation='sigmoid')) # only difference is the activation finction
    model.compile(SGD(lr=0.5), 'binary_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
model= KerasClassifier(build_fn=build_logistic, epochs=25) # behaves like a sklearn model, to validate with the sklearn
                                                           # libraries

In [38]:
from sklearn.model_selection import cross_val_score, KFold

In [17]:
'''k folded training evaluation of the model'''

cv = KFold(3,shuffle=True)

In [18]:
scores = cross_val_score(model,X,y,cv=cv)

Epoch 1/25
66/66 [==============================] - 0s 3ms/step - loss: 0.8992 - acc: 0.6061
Epoch 2/25
66/66 [==============================] - 0s 232us/step - loss: 0.5478 - acc: 0.6667
Epoch 3/25
66/66 [==============================] - 0s 262us/step - loss: 0.4969 - acc: 0.8182
Epoch 4/25
66/66 [==============================] - 0s 287us/step - loss: 0.6055 - acc: 0.6364
Epoch 5/25
66/66 [==============================] - 0s 166us/step - loss: 0.5553 - acc: 0.7121
Epoch 6/25
66/66 [==============================] - 0s 273us/step - loss: 0.5016 - acc: 0.8030
Epoch 7/25
66/66 [==============================] - 0s 227us/step - loss: 0.4491 - acc: 0.8333
Epoch 8/25
66/66 [==============================] - 0s 210us/step - loss: 0.4450 - acc: 0.8485
Epoch 9/25
66/66 [==============================] - 0s 139us/step - loss: 0.4471 - acc: 0.8182
Epoch 10/25
66/66 [==============================] - 0s 211us/step - loss: 0.4338 - acc: 0.8333
Epoch 11/25
66/66 [==============================] 

In [19]:
scores

array([0.76470588, 0.81818182, 0.78787879])

# confusion matrix

In [27]:
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [24]:
confusion_matrix(y,y_class_pred)

array([[40, 10],
       [ 7, 43]])

In [26]:
f1_score(y, y_class_pred)

0.8349514563106797

In [29]:
print(classification_report(y,y_class_pred))

             precision    recall  f1-score   support

          0       0.85      0.80      0.82        50
          1       0.81      0.86      0.83        50

avg / total       0.83      0.83      0.83       100



# Employee churn

In [12]:
df = pd.read_csv('../data/HR_comma_sep.csv')

In [13]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [14]:
'''Base line estimation'''

df.left.value_counts()/len(df)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


0    0.761917
1    0.238083
Name: left, dtype: float64

In [15]:
'''normalizing the total number of hours'''

df['average_montly_hours'] = df['average_montly_hours']/100.0

In [16]:
'''categorical conversion'''

df_dummies = pd.get_dummies(df[['sales', 'salary']])

In [17]:
X = pd.concat([df[['satisfaction_level', 'last_evaluation', 'number_project',
                   'average_montly_hours', 'time_spend_company', 
                   'Work_accident','promotion_last_5years']], df_dummies], axis=1).values

In [18]:
y=df.left.values

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [21]:
'''neural network model'''

model = Sequential()
model.add(Dense(8,input_dim=20, activation='sigmoid'))
model.add(Dense(4, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(Adam(lr=0.8), 'binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [23]:
# y_test_pred = model.predict(X_test) # for porbabilities
y_test_pred = model.predict_classes(X_test)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
confusion_matrix(y_test, y_test_pred)

array([[2236,    0],
       [ 764,    0]])

In [26]:
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.75      1.00      0.85      2236
          1       0.00      0.00      0.00       764

avg / total       0.56      0.75      0.64      3000



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
from keras.wrappers.scikit_learn import  KerasClassifier
def build_logistic():
    model = Sequential()
    model.add(Dense(1,input_shape=(20,), activation='sigmoid')) # only difference is the activation finction
    model.compile(Adam(lr=0.5), 'binary_crossentropy', metrics=['accuracy'])
    
    return model

In [37]:
model = KerasClassifier(build_fn=build_logistic, epochs=10)


In [ ]:
'''accuracy scores for the different folded data'''

cv = KFold(5, shuffle=True)
scores = cross_val_score(model,X,y,cv=cv)

In [40]:
scores

array([0.80633333, 0.787     , 0.75966667, 0.778     , 0.78926309])